In [1]:
def import_predef_funcs():
    import pandas as pd
    import seaborn as sns
    import pylab
    import numpy as np
    import sklearn
    from sklearn import linear_model
    import sklearn.preprocessing as preprocessing
    import sklearn.metrics as metrics
    import matplotlib.pyplot as plt
    from statsmodels.stats import proportion

    import pandas as pd
    from sklearn.datasets import fetch_openml
    from sklearn.metrics import accuracy_score, precision_score, recall_score
    from sklearn.tree import DecisionTreeClassifier


    from sklearn.metrics import balanced_accuracy_score, roc_auc_score,accuracy_score

    
    from sklearn.model_selection import train_test_split

    # Fairlearn algorithms and utils
    from fairlearn.postprocessing import ThresholdOptimizer
    from fairlearn.reductions import GridSearch, EqualizedOdds

    # Metrics
    from fairlearn.metrics import (
        MetricFrame,
        selection_rate, demographic_parity_difference, demographic_parity_ratio,
        false_positive_rate, false_negative_rate,
        false_positive_rate_difference, false_negative_rate_difference,
        equalized_odds_difference, count)

In [2]:
# create_table(train_data,train_labels,test_data,test_labels,
#                  model,A,A_,A_str_train,A_str_test,
#                  constraint,dr_feat)
def create_table(keep_feat,train_data,train_labels,test_data,test_labels,model,A,A_,A_str_train,A_str_test,constraint,dr_feat,dataType):
#################################### Imports needed ###################################################
    import pandas as pd
    import seaborn as sns
    import pylab
    import numpy as np
    import sklearn
    from sklearn import linear_model
    import sklearn.preprocessing as preprocessing
    import sklearn.metrics as metrics
    import matplotlib.pyplot as plt
    from statsmodels.stats import proportion

    import pandas as pd
    from sklearn.datasets import fetch_openml
    from sklearn.metrics import accuracy_score, precision_score, recall_score
    from sklearn.tree import DecisionTreeClassifier


    from sklearn.metrics import balanced_accuracy_score, roc_auc_score,accuracy_score
    import warnings
    warnings.filterwarnings('ignore')
    
    from sklearn.model_selection import train_test_split

    # Fairlearn algorithms and utils
    from fairlearn.postprocessing import ThresholdOptimizer
    from fairlearn.reductions import GridSearch, EqualizedOdds
    from fairlearn.reductions import ExponentiatedGradient,EqualizedOdds,DemographicParity

    # Metrics
    from fairlearn.metrics import (
        MetricFrame,
        selection_rate, demographic_parity_difference, demographic_parity_ratio,
        false_positive_rate, false_negative_rate,
        false_positive_rate_difference, false_negative_rate_difference,
        equalized_odds_difference, count)

################## End of imports & start of actual function code ##################################

    abbr_feat = {"Age":"Age","Race":"Race","Sex":"Sex","Workclass":"WrkCls","fnlwgt":"fnlwgt","Education-Num":"EdNum","Marital Status":"MarSts",
                 "Occupation":"Occpn","Relationship":"Rltshp","Capital Gain":"CapGn","Capital Loss":"CapLs",
                 "Hours per week":"HrPWk","Country":"Cntry", "allFeat":"allFeat"}
    
    if constraint == 'equalized_odds':
        name = "EO"
        const = EqualizedOdds()
    if constraint == 'demographic_parity':
        name = "DP"
        const = DemographicParity()
        
        
    valid_features = ["Workclass", "fnlwgt", "Education-Num", "Marital Status",
            "Occupation", "Relationship", "Capital Gain", "Capital Loss",
            "Hours per week", "Country","Age","Race","Sex"] 

    rem_feat = [] # need to store all the one hot coded version of the feature to drop
    
    for things in valid_features:
        if things not in keep_feat:
            for cols in train_data.columns:
                if things in cols:
                    rem_feat.append(cols)
    if dr_feat is not None:
        for cols in train_data.columns:
            if dr_feat in cols:
                rem_feat.append(cols)
    else: 
        dr_feat = "allFeat"
    
            
    # drop the feature we dont want from train and test
    train_data = train_data.drop(columns = rem_feat)
#     print(train_data.columns)
    test_data = test_data.drop(columns = rem_feat)
    


    import lightgbm as lgb
    from sklearn.calibration import CalibratedClassifierCV

    lgb_params = {
        'objective' : 'binary',
        'metric' : 'cross_entropy', 
        'learning_rate': 0.1,
        'num_leaves' : 10,
        'max_depth' : 3
    }

    model = lgb.LGBMClassifier(**lgb_params)
    model.fit(train_data, train_labels)
    # Scores on test set
    test_scores = model.predict_proba(test_data)[:, 1]
    test_pred = model.predict(test_data)
    # Train AUC
    roc_auc_score(train_labels, model.predict_proba(train_data)[:, 1])
    # ACCURACY
    accuracy_score(test_labels, test_pred)
    # Predictions (0 or 1) on test set
    test_preds = (test_scores >= np.mean(train_labels)) * 1
    
    from sklearn.model_selection import train_test_split

    # Fairlearn algorithms and utils
    from fairlearn.postprocessing import ThresholdOptimizer
    from fairlearn.reductions import GridSearch, EqualizedOdds

    # Metrics
    from fairlearn.metrics import (
        MetricFrame,
        selection_rate, demographic_parity_difference, demographic_parity_ratio,
        false_positive_rate, false_negative_rate,
        false_positive_rate_difference, false_negative_rate_difference,
        equalized_odds_difference, count)
    
        # Helper functions
    def get_metrics_df(models_dict, y_true, group):
        metrics_dict = {
            "Overall selection rate": (
                lambda x: selection_rate(y_true, x), True),
            "Demographic parity difference": (
                lambda x: demographic_parity_difference(y_true, x, sensitive_features=group), True),
            "False positive rate difference": (
                lambda x: false_positive_rate_difference(y_true, x, sensitive_features=group), True),
            "False negative rate difference": (
                lambda x: false_negative_rate_difference(y_true, x, sensitive_features=group), True),
            "Equalized odds difference": (
                lambda x: equalized_odds_difference(y_true, x, sensitive_features=group), True),
#             "Overall AUC": (
#                 lambda x: roc_auc_score(y_true, x), False),
            "Accuracy": (
             lambda x: accuracy_score(y_true, x), True),
            "AUC difference": (
                lambda x: MetricFrame(metrics=roc_auc_score, y_true=y_true, y_pred=x, sensitive_features=group).difference(method='between_groups'), False),
        }
        df_dict = {}
        for metric_name, (metric_func, use_preds) in metrics_dict.items():
            df_dict[metric_name] = [metric_func(preds) if use_preds else metric_func(scores) 
                                    for model_name, (preds, scores) in models_dict.items()]
        return pd.DataFrame.from_dict(df_dict, orient="index", columns=models_dict.keys())
    
    if dataType == "threshold_opt" or dataType =="unmitigated":
        postprocess_est = ThresholdOptimizer(
            estimator = model,
            constraints= constraint,
            prefit=True)

        balanced_idx1 = train_data[train_labels==1].index
        pp_train_idx = balanced_idx1.union(train_labels[train_labels==0].sample(n=balanced_idx1.size, random_state=1234).index)
        df_train_balanced = train_data.loc[pp_train_idx, :]
        Y_train_balanced = train_labels.loc[pp_train_idx]
        A_train_balanced = A_.loc[pp_train_idx]


        postprocess_est.fit(df_train_balanced, Y_train_balanced, sensitive_features=A_train_balanced)

        postprocess_preds = postprocess_est.predict(test_data, sensitive_features= A)


        models_dict = {"Unmitigated": (test_preds, test_scores),
                      "ThresholdOptimizer": (postprocess_preds, postprocess_preds)}
        output = get_metrics_df(models_dict, test_labels, A_str_test)
        output.rename(columns={'Unmitigated': name+"_Unmit:"+abbr_feat[dr_feat], 
                               'ThresholdOptimizer': name+"_ThOpt:"+abbr_feat[dr_feat]}, inplace=True)
        
    if dataType == "Exponentiated_grad":
        from fairlearn.reductions import ExponentiatedGradient,EqualizedOdds,DemographicParity
#         from xgboost import XGBClassifier
#         from sklearn.ensemble import RandomForestClassifier
#         clf = RandomForestClassifier(n_estimators=10)
#         xgc = XGBClassifier(verbosity=0)

        eg = ExponentiatedGradient(estimator= model, constraints= const,max_iter=2)
        eg.fit(train_data,train_labels, sensitive_features=A_)
        eg_test_preds = eg.predict(test_data)

        models_dict = {"Unmitigated": (test_preds, test_scores),
                      "ExponentiatedGrad": (eg_test_preds,eg_test_preds)}
        output = (get_metrics_df(models_dict, test_labels, A_str_test))
        output.rename(columns={'Unmitigated': name+"_Unmit:"+abbr_feat[dr_feat], 
                       'ExponentiatedGrad': name+"_ExGrd:"+abbr_feat[dr_feat]}, inplace=True)



    return output
    








In [3]:
def compute_Ascore(output,alpha,constraint,dataType = "unmitigated"):
    
#################################### Imports needed ###################################################
    import pandas as pd
    import seaborn as sns
    import pylab
    import numpy as np
    import sklearn
    from sklearn import linear_model
    import sklearn.preprocessing as preprocessing
    import sklearn.metrics as metrics
    import matplotlib.pyplot as plt
    from statsmodels.stats import proportion

    import pandas as pd
    from sklearn.datasets import fetch_openml
    from sklearn.metrics import accuracy_score, precision_score, recall_score
    from sklearn.tree import DecisionTreeClassifier


    from sklearn.metrics import balanced_accuracy_score, roc_auc_score,accuracy_score
    import warnings
    warnings.filterwarnings('ignore')

    
    from sklearn.model_selection import train_test_split

    # Fairlearn algorithms and utils
    from fairlearn.postprocessing import ThresholdOptimizer
    from fairlearn.reductions import GridSearch, EqualizedOdds

    # Metrics
    from fairlearn.metrics import (
        MetricFrame,
        selection_rate, demographic_parity_difference, demographic_parity_ratio,
        false_positive_rate, false_negative_rate,
        false_positive_rate_difference, false_negative_rate_difference,
        equalized_odds_difference, count)

################## End of imports & start of actual function code ##################################
    if constraint == 'equalized_odds':
            name = "EO"
    else:
            name = "DP"

    if dataType == "unmitigated":
        col_len = len(output.columns)
        del_AUC_unmit = ( output.iloc[5,0] - output.iloc[5,np.arange(0,col_len-1,2)] ) / output.iloc[5,0]
        if name == "EO":
            del_EO_unmit = ( output.iloc[4,0] - output.iloc[4,np.arange(0,col_len-1,2)] ) / output.iloc[4,0]
        if name == "DP":
            del_EO_unmit = ( output.iloc[1,0] - output.iloc[1,np.arange(0,col_len-1,2)] ) / output.iloc[1,0]

        del_AUC_unmit = del_AUC_unmit.drop(labels = name+"_Unmit:allFeat", axis = 0)
        del_EO_unmit = del_EO_unmit.drop(labels = name+"_Unmit:allFeat", axis = 0)
    
    if dataType == "threshold_opt":
        col_len = len(output.columns)
        del_AUC_unmit = ( output.iloc[5,1] - output.iloc[5,np.arange(1,col_len+1,2)] ) / output.iloc[5,1]
        if name == "EO":
            del_EO_unmit = ( output.iloc[4,1] - output.iloc[4,np.arange(1,col_len+1,2)] ) / output.iloc[4,1]
        if name == "DP":
            del_EO_unmit = ( output.iloc[1,1] - output.iloc[1,np.arange(1,col_len+1,2)] ) / output.iloc[1,1]
            
        del_AUC_unmit = del_AUC_unmit.drop(labels = name+"_ThOpt:allFeat", axis = 0)
        del_EO_unmit = del_EO_unmit.drop(labels = name+"_ThOpt:allFeat", axis = 0)
        
    if dataType =="Exponentiated_grad":
        col_len = len(output.columns)
        del_AUC_unmit = ( output.iloc[5,1] - output.iloc[5,np.arange(1,col_len+1,2)] ) / output.iloc[5,1]
        if name == "EO":
            del_EO_unmit = ( output.iloc[4,1] - output.iloc[4,np.arange(1,col_len+1,2)] ) / output.iloc[4,1]
        if name == "DP":
            del_EO_unmit = ( output.iloc[1,1] - output.iloc[1,np.arange(1,col_len+1,2)] ) / output.iloc[1,1]
            
        del_AUC_unmit = del_AUC_unmit.drop(labels = name+"_ExGrd:allFeat", axis = 0)
        del_EO_unmit = del_EO_unmit.drop(labels = name+"_ExGrd:allFeat", axis = 0)
    
#     alpha = np.linspace(0,0.5,num=20)

    A_score = np.zeros((alpha.shape[0],del_AUC_unmit.shape[0]))
    for i in np.arange(0,len(alpha)):
        A_score[i,:] =(1.0-alpha[i]) * del_AUC_unmit + alpha[i] * (-del_EO_unmit)
     
    if dataType =="Exponentiated_grad":
        df_col = output.columns.drop([name+"_Unmit:allFeat",name+"_ExGrd:allFeat"]) 
    if dataType == "threshold_opt":
        df_col = output.columns.drop([name+"_Unmit:allFeat",name+"_ThOpt:allFeat"]) 
    if dataType == "unmitigated":
        df_col = output.columns.drop([name+"_Unmit:allFeat",name+"_ThOpt:allFeat"]) 
        
    A_score_df = pd.DataFrame(A_score, columns = del_AUC_unmit.index ,index=alpha)
    
    return A_score,A_score_df
    